# AutoGen - Stocks Prices demo

AutoGen is a framework that enables the development of LLM applications using multiple agents that can converse with each other to solve tasks. AutoGen agents are customizable, conversable, and seamlessly allow human participation. They can operate in various modes that employ combinations of LLMs, human inputs, and tools.

<img src="https://github.com/microsoft/autogen/blob/main/website/static/img/autogen_agentchat.png?raw=true">

> https://github.com/microsoft/autogen

In [ ]:
%pip install yfinance

In [ ]:
%pip install pyautogen

In [1]:
from typing import Literal

from pydantic import BaseModel, Field
from typing_extensions import Annotated
from autogen.cache import Cache


C:\Users\faismali\AppData\Local\anaconda3\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


In [2]:
import autogen
import datetime
import openai
import os
import pandas as pd
import requests
import yfinance as yf
import sys

from autogen import AssistantAgent, UserProxyAgent
from dotenv import load_dotenv

In [3]:
sys.version

'3.11.4 | packaged by Anaconda, Inc. | (main, Jul  5 2023, 13:38:37) [MSC v.1916 64 bit (AMD64)]'

In [4]:
print(f"Today is {datetime.datetime.today().strftime('%d-%b-%Y %H:%M:%S')}")

Today is 29-Apr-2024 08:51:14


## Configuration

In [5]:
!ls OAI_CONFIG_LIST -lh

'ls' is not recognized as an internal or external command,
operable program or batch file.


In [6]:
config_list = autogen.config_list_from_json(
    "OAI_CONFIG_LIST",
    filter_dict={"tags": ["3.5-tool"]}
)

In [7]:
print("You have this required model:", config_list[0]["model"])

You have this required model: gpt-35-turbo


## Agents

In [8]:
llm_config = {
    "config_list": config_list,
    "timeout": 120,
}


In [9]:
chatbot = autogen.AssistantAgent(
    name="chatbot",
    system_message="For currency exchange tasks, only use the functions you have been provided with. Reply TERMINATE when the task is done.",
    llm_config=llm_config,
)

In [10]:
# create a UserProxyAgent instance named "user_proxy"
user_proxy = autogen.UserProxyAgent(
    name="user_proxy",
    is_termination_msg=lambda x: x.get("content", "") and x.get("content", "").rstrip().endswith("TERMINATE"),
    human_input_mode="NEVER",
    max_consecutive_auto_reply=10,
    code_execution_config=False
)

In [11]:
CurrencySymbol = Literal["USD", "EUR"]


## Functions

In [12]:
def exchange_rate(base_currency: CurrencySymbol, quote_currency: CurrencySymbol) -> float:
    if base_currency == quote_currency:
        return 1.0
    elif base_currency == "USD" and quote_currency == "EUR":
        return 1 / 1.1
    elif base_currency == "EUR" and quote_currency == "USD":
        return 1.1
    else:
        raise ValueError(f"Unknown currencies {base_currency}, {quote_currency}")


In [13]:
@user_proxy.register_for_execution()
@chatbot.register_for_llm(description="Currency exchange calculator.")
def currency_calculator(
    base_amount: Annotated[float, "Amount of currency in base_currency"],
    base_currency: Annotated[CurrencySymbol, "Base currency"] = "USD",
    quote_currency: Annotated[CurrencySymbol, "Quote currency"] = "EUR",
) -> str:
    quote_amount = exchange_rate(base_currency, quote_currency) * base_amount
    return f"{quote_amount} {quote_currency}"

In [14]:
chatbot.llm_config["tools"]



[{'type': 'function',
  'function': {'description': 'Currency exchange calculator.',
   'name': 'currency_calculator',
   'parameters': {'type': 'object',
    'properties': {'base_amount': {'type': 'number',
      'description': 'Amount of currency in base_currency'},
     'base_currency': {'enum': ['USD', 'EUR'],
      'type': 'string',
      'default': 'USD',
      'description': 'Base currency'},
     'quote_currency': {'enum': ['USD', 'EUR'],
      'type': 'string',
      'default': 'EUR',
      'description': 'Quote currency'}},
    'required': ['base_amount']}}}]

In [15]:
assert user_proxy.function_map["currency_calculator"]._origin == currency_calculator



In [16]:
with Cache.disk() as cache:
    # start the conversation
    res = user_proxy.initiate_chat(
        chatbot, message="How much is 123.45 USD in EUR?", summary_method="reflection_with_llm", cache=cache
    )

user_proxy (to chatbot):

How much is 123.45 USD in EUR?

--------------------------------------------------------------------------------
chatbot (to user_proxy):

***** Suggested tool call (call_bQXNdOKojJonnVIpIoLVKKug): currency_calculator *****
Arguments: 
{
  "base_amount": 123.45,
  "base_currency": "USD",
  "quote_currency": "EUR"
}
************************************************************************************

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING FUNCTION currency_calculator...
user_proxy (to chatbot):

user_proxy (to chatbot):

***** Response from calling tool (call_bQXNdOKojJonnVIpIoLVKKug) *****
112.22727272727272 EUR
**********************************************************************

--------------------------------------------------------------------------------
chatbot (to user_proxy):

123.45 USD is equivalent to 112.23 EUR.

------------------------------------------------------------------------

# Group Chat

In [21]:
config_list = autogen.config_list_from_json(
    "OAI_CONFIG_LIST",
    filter_dict={
        "model": ["gpt-4", "gpt-4-0314", "gpt4", "gpt-4-32k", "gpt-4-32k-0314", "gpt-4-32k-v0314"],
    },
)

In [22]:
print("You have this required model:", config_list[0]["model"])

You have this required model: gpt-4-32k


In [25]:
llm_config = {"config_list": config_list, "cache_seed": 42}
user_proxy = autogen.UserProxyAgent(
    name="User_proxy",
    system_message="A human admin.",
    code_execution_config={
        "last_n_messages": 2,
        "work_dir": "groupchat",
        "use_docker": True,
    },  # Please set use_docker=True if docker is available to run the generated code. Using docker is safer than running the generated code directly.
    human_input_mode="TERMINATE",
)
coder = autogen.AssistantAgent(
    name="Coder",
    llm_config=llm_config,
)
pm = autogen.AssistantAgent(
    name="Product_manager",
    system_message="Creative in software product ideas.",
    llm_config=llm_config,
)
groupchat = autogen.GroupChat(agents=[user_proxy, coder, pm], messages=[], max_round=12)
manager = autogen.GroupChatManager(groupchat=groupchat, llm_config=llm_config)

In [26]:
user_proxy.initiate_chat(
    manager, message="Find a latest paper about gpt-4 on arxiv and find its potential applications in software."
)
# type exit to terminate the chat

User_proxy (to chat_manager):

Find a latest paper about gpt-4 on arxiv and find its potential applications in software.

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
User_proxy (to chat_manager):



--------------------------------------------------------------------------------
Coder (to chat_manager):

To perform this task, we first need to search for the latest paper related to "GPT-4" on the arXiv platform and then mine the paper for any potential applications in software. We will use python to handle both of these tasks. 

The first step would be to call arXiv API to search for the papers related to "GPT-4". We'll scrape the first few search results and find the latest one. For this, we'll use the 'arxiv' Python library. If the 'arxiv' library is not installed, please run the following command to install it: 

```python
!pip install arxiv
```
Once completed, execute the following python script:

```python
import ar


--------------------------------------------------------------------------------
Coder (to chat_manager):

I apologize for the inconvenience, it seems like there has been a change in the arxiv library that I overlooked. It should have been `search.results` instead of `search.get()`. 

Please try running the corrected code below:

```python
import subprocess
import sys

def install(package):
    subprocess.check_call([sys.executable, "-m", "pip", "install", package])

try:
    import arxiv 
except ImportError:
    install('arxiv')
    import arxiv 

# We will search for "GPT-4" in all categories
search = arxiv.Search(
    query="GPT-4",
    max_results=50,
)

# We will only focus on papers that were uploaded this year
papers_2022 = [i for i in search.results() if i.published.year == 2022]

# Sort by published date
papers_2022.sort(key=lambda x: x.published, reverse=True)

# Print the latest one
latest_paper = papers_2022[0]
print(f"The title of the latest paper is '{latest_paper.title}


--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
User_proxy (to chat_manager):



--------------------------------------------------------------------------------


ChatResult(chat_id=None, chat_history=[{'content': 'Find a latest paper about gpt-4 on arxiv and find its potential applications in software.', 'role': 'assistant'}, {'content': '', 'role': 'assistant'}, {'content': 'To perform this task, we first need to search for the latest paper related to "GPT-4" on the arXiv platform and then mine the paper for any potential applications in software. We will use python to handle both of these tasks. \n\nThe first step would be to call arXiv API to search for the papers related to "GPT-4". We\'ll scrape the first few search results and find the latest one. For this, we\'ll use the \'arxiv\' Python library. If the \'arxiv\' library is not installed, please run the following command to install it: \n\n```python\n!pip install arxiv\n```\nOnce completed, execute the following python script:\n\n```python\nimport arxiv \n\n#We will search for "GPT-4" in all categories\nsearch = arxiv.Search(\n   query = "GPT-4",\n   max_results = 50,\n)\n\n#We will only